<a href="https://colab.research.google.com/github/AnuragSingh7292/Remote-sensing-image-retrieval-deep-learning-/blob/main/perProcessing(NLM%20and%20HE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# After running, follow the authentication steps


Mounted at /content/drive


In [ ]:
 import zipfile
import os

# Define the paths
zip_path = "/content/drive/MyDrive/Images.zip"  # Update with your actual file path
extract_path = "/content/UC_Merced_LandUse"  # Temporary extraction path

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Extracted categories:", os.listdir(extract_path))  # Should list 21 folders


Extracted categories: ['Images']


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_PCA_PrePro_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'avg_pool'
fc6_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

import cv2
import numpy as np

def preprocess_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)  # Load image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, target_size)  # Resize image

    # Intensity normalization (scale to 0-1 range)
    img = img.astype(np.float32) / 255.0

    # Convert to YCrCb and apply Histogram Equalization on Y channel
    ycrcb = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    y = cv2.equalizeHist((y * 255).astype(np.uint8))  # Apply HE

    # Merge back and convert to RGB
    ycrcb = cv2.merge((y, cr.astype(np.uint8), cb.astype(np.uint8)))
    img = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2RGB)

    return img



# Function to extract features
def extract_features(model, img_path):
    img = preprocess_image(img_path)  # Apply preprocessing
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)  # Apply ResNet preprocessing
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
fc6_features_101, labels = [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                fc6_feat = extract_features(fc6_layer_101, img_path)
                fc6_features_101.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
fc6_features_101 = np.array(fc6_features_101)

# Save raw extracted features
np.save(os.path.join(save_dir, "resnet101_fc6_pre1_features.npy"), fc6_features_101)

# Apply PCA to reduce dimensions to (2100, 32)
pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save reduced features
np.save(os.path.join(save_dir, "resnet101_fc6_32_pre1_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labels_pre.npy"), labels)

print("✅ Preprocessing applied & features extracted successfully for ResNet101! (Shape: (2100, 32))")


✅ Preprocessing applied & features extracted successfully for ResNet101! (Shape: (2100, 32))


In [ ]:
print(fc6_features_101.shape)
print(fc6_reduced_101.shape)


(2100, 2048)
(2100, 32)


In [ ]:
!pip install rembg pillow
!apt-get install -y libglib2.0-0

!pip install rembg onnxruntime pillow




import os
import numpy as np
import cv2
import tensorflow as tff
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA
from rembg import remove
from PIL import Image

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_PCA_PrePro_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'avg_pool'
fc6_layer_101 = tff.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

def preprocess_image_with_bg_removal(img_path, target_size=(224, 224)):
    # Load image
    input_image = Image.open(img_path)

    # Remove background
    output_image = remove(input_image)

    # Convert the image to a numpy array for further processing
    img = np.array(output_image)

    # If the image has an alpha channel (RGBA), replace transparent areas with white
    if img.shape[2] == 4:
        # Extract the alpha channel
        alpha_channel = img[:, :, 3]
        # Create a white background
        white_background = np.ones_like(img[:, :, :3]) * 255  # All white

        # Replace transparent areas with white
        img = np.dstack((white_background, alpha_channel))

        # Convert to RGB (discard alpha channel)
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

    # Resize image
    img = cv2.resize(img, target_size)  # Resize image

    return img

# Function to extract features
def extract_features(model, img_path):
    img = preprocess_image_with_bg_removal(img_path)  # Apply preprocessing with bg removal
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)  # Apply ResNet preprocessing
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
fc6_features_101, labels = [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                fc6_feat = extract_features(fc6_layer_101, img_path)
                fc6_features_101.append(fc6_feat)
                labels.append(idx)

# Convert lists to arrays
fc6_features_101 = np.array(fc6_features_101)

# Save raw extracted features
np.save(os.path.join(save_dir, "resnet101_fc6_pre2_features.npy"), fc6_features_101)

# Apply PCA to reduce dimensions to (2100, 32)
pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save reduced features
np.save(os.path.join(save_dir, "resnet101_fc6_32_pre2_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labels_pre.npy"), labels)

print("✅ Preprocessing applied & features extracted successfully for ResNet101! (Shape: (2100, 32))")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglib2.0-0 is already the newest version (2.72.4-0ubuntu2.4).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [ ]:
# !pip install rembg pillow
# !apt-get install -y libglib2.0-0

# !pip install rembg onnxruntime pillow
# !pip uninstall -y tensorflow
# !pip install tensorflow --upgrade
# !pip install rembg pillow onnxruntime


import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA
from rembg import remove
from PIL import Image

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_PCA_PrePro_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'avg_pool'
fc6_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

# Function to preprocess images (with background removal)
def preprocess_image(img_path, target_size=(224, 224)):
    # Load image using PIL
    input_image = Image.open(img_path)

    # Remove background
    output_image = remove(input_image)

    # Convert to NumPy array
    img = np.array(output_image)

    # If image has an alpha channel (RGBA), replace transparency with white
    if img.shape[-1] == 4:
        alpha = img[:, :, 3]  # Extract alpha channel
        white_bg = np.ones_like(img[:, :, :3]) * 255  # White background
        img = np.where(alpha[:, :, None] > 0, img[:, :, :3], white_bg)

    # Convert to OpenCV format (ensuring 3 channels)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB) if img.shape[-1] == 4 else img

    # Resize to match ResNet input size
    img = cv2.resize(img, target_size)

    # Normalize pixel values (0-1 range)
    img = img.astype(np.float32) / 255.0

    return img

# Function to extract features
def extract_features(model, img_path):
    img = preprocess_image(img_path)  # Apply preprocessing
    img_array = np.expand_dims(img, axis=0)
    img_array = preprocess_input(img_array)  # Apply ResNet preprocessing
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
fc6_features_101, labels = [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)

                try:
                    # Extract features after preprocessing (including background removal)
                    fc6_feat = extract_features(fc6_layer_101, img_path)
                    fc6_features_101.append(fc6_feat)
                    labels.append(idx)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")

# Convert lists to arrays
fc6_features_101 = np.array(fc6_features_101)

# Save raw extracted features
np.save(os.path.join(save_dir, "resnet101_fc6_pre2_features.npy"), fc6_features_101)

# Apply PCA to reduce dimensions to (2100, 32)
pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save reduced features
np.save(os.path.join(save_dir, "resnet101_fc6_32_pre2_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labels_pre.npy"), labels)

print("✅ Background removed & features extracted successfully for ResNet101! (Shape: (2100, 32))")


179648224/179648224 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 37.6GB/s]


✅ Background removed & features extracted successfully for ResNet101! (Shape: (2100, 32))


In [ ]:
print( fc6_reduced_101.shape)

NLM

In [ ]:

!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA
from skimage import io, img_as_float, img_as_ubyte
from skimage.restoration import denoise_nl_means, estimate_sigma
import cv2

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_PCA_PrePro_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'avg_pool' only
fc6_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

# Function to apply NLM and prepare image
def preprocess_image_nlm(img_path, target_size=(224, 224)):
    img_gray = img_as_float(io.imread(img_path, as_gray=True))
    sigma_est = estimate_sigma(img_gray, average_sigmas=True)

    denoised = denoise_nl_means(img_gray, h=1.15 * sigma_est, fast_mode=True,
                                patch_size=5, patch_distance=6)

    denoised_uint8 = img_as_ubyte(denoised)
    denoised_rgb = cv2.cvtColor(denoised_uint8, cv2.COLOR_GRAY2RGB)
    denoised_resized = cv2.resize(denoised_rgb, target_size)

    img_array = image.img_to_array(denoised_resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    return img_array

# Feature extraction function
def extract_features(model, img_path):
    img_array = preprocess_image_nlm(img_path)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
fc6_features_101, labels = [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                fc6_feat = extract_features(fc6_layer_101, img_path)
                fc6_features_101.append(fc6_feat)
                labels.append(idx)

# Convert to array
fc6_features_101 = np.array(fc6_features_101)

# Save raw features
np.save(os.path.join(save_dir, "resnet101_fc6_fc6_NLM_features.npy"), fc6_features_101)

# Apply PCA
pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save reduced features
np.save(os.path.join(save_dir, "resnet101_fc6_32_NLM_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labelNLM.npy"), labels)

print("✅ FC6 features extracted using ResNet101 + NLM preprocessing and saved successfully! (Shape: (2100, 32))")


✅ FC6 features extracted using ResNet101 + NLM preprocessing and saved successfully! (Shape: (2100, 32))


In [ ]:
print(fc6_reduced_101.shape)

(2100, 32)


NLM and  HE

In [ ]:

!pip install PyWavelets
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.decomposition import PCA
from skimage import io, img_as_float, img_as_ubyte
from skimage.restoration import denoise_nl_means, estimate_sigma
import cv2

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_PCA_PrePro_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load ResNet101 model with pre-trained weights
base_model_101 = ResNet101(weights='imagenet', include_top=True)

# Extracting features from 'avg_pool' only
fc6_layer_101 = tf.keras.Model(inputs=base_model_101.input,
                               outputs=base_model_101.get_layer('avg_pool').output)

# Function to apply NLM and prepare image
def preprocess_image_nlm_he(img_path, target_size=(224, 224)):
    # Load grayscale and convert to float
    img_gray = img_as_float(io.imread(img_path, as_gray=True))

    # Estimate sigma
    sigma_est = estimate_sigma(img_gray, average_sigmas=True)

    # Denoise using NLM
    denoised = denoise_nl_means(img_gray, h=1.15 * sigma_est, fast_mode=True,
                                patch_size=5, patch_distance=6)

    # Convert to uint8 for histogram equalization
    denoised_uint8 = img_as_ubyte(denoised)

    # Apply histogram equalization
    equalized = cv2.equalizeHist(denoised_uint8)

    # Convert grayscale equalized image to RGB
    equalized_rgb = cv2.cvtColor(equalized, cv2.COLOR_GRAY2RGB)

    # Resize to 224x224
    resized = cv2.resize(equalized_rgb, target_size)

    # Prepare for model input
    img_array = image.img_to_array(resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    return img_array


# Feature extraction function
def extract_features(model, img_path):
    img_array = preprocess_image_nlm_he(img_path)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
fc6_features_101, labels = [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)
                fc6_feat = extract_features(fc6_layer_101, img_path)
                fc6_features_101.append(fc6_feat)
                labels.append(idx)

# Convert to array
fc6_features_101 = np.array(fc6_features_101)

# Save raw features
np.save(os.path.join(save_dir, "resnet101_fc6_fc6_NLM_HE_features.npy"), fc6_features_101)

# Apply PCA
pca_fc6_101 = PCA(n_components=32)
fc6_reduced_101 = pca_fc6_101.fit_transform(fc6_features_101)

# Save reduced features
np.save(os.path.join(save_dir, "resnet101_fc6_32_NLM_HE_features.npy"), fc6_reduced_101)
np.save(os.path.join(save_dir, "labelNLM.npy"), labels)

print("✅ FC6 features extracted using ResNet101 + NLM preprocessing and saved successfully! (Shape: (2100, 32))")


✅ FC6 features extracted using ResNet101 + NLM preprocessing and saved successfully! (Shape: (2100, 32))
